In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
import  pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU, Attention, Input, Layer, Flatten, Lambda
from tensorflow.keras.callbacks import EarlyStopping
from nltk.corpus import gutenberg
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [2]:
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

In [3]:
with open('hamlet.txt','r') as file:
    text = file.read().lower()


tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words


4818

In [4]:
#Creating input sequence
input_sequences = []
for line in text.split('\n'):
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequences = token_list[: i +1]
    input_sequences.append(n_gram_sequences)

In [5]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [6]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
input_sequences


array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [7]:
x,y = input_sequences[:, :-1], input_sequences[:, -1]

In [8]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [11]:
inputs = Input(shape=(max_sequence_len - 1,))
embedding_layer = Embedding(total_words, 100)(inputs)
lstm_out = LSTM(150, return_sequences=True)(embedding_layer)
dropout_out = Dropout(0.2)(lstm_out)


query_vector = Lambda(lambda x: tf.expand_dims(x[:, -1, :], axis=1),
                      output_shape=(1, 150),
                      name='query_extractor')(dropout_out)

value_sequence = dropout_out
key_sequence = dropout_out


attention_output = Attention()([query_vector, value_sequence, key_sequence])


flattened_attention = Flatten()(attention_output)


output = Dense(total_words, activation='softmax')(flattened_attention)

model_attention = tf.keras.Model(inputs=inputs, outputs=output)


model_attention.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_attention.summary()

print("\nTraining Model with Attention Layer...")
history_attention = model_attention.fit(x_train, y_train,
                                        epochs=10,
                                        batch_size=32,
                                        validation_data=(x_test, y_test),
                                        callbacks=[early_stopping],
                                        verbose=1)


loss_attention, accuracy_attention = model_attention.evaluate(x_test, y_test, verbose=0)
print(f"\nModel with Attention Test Accuracy: {accuracy_attention:.4f}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 13, 100)   │    481,800 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 13, 150)   │    150,600 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 13, 150)   │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ query_extractor     │ (None, 1, 150)    │          0 │ dropout[0][0]     │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention           │ (None, 1, 150)    │          0 │ query_extractor[… │
│ (Attention)         │                   │            │ dropout[0][0],    │
│                     │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 150)       │          0 │ attention[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 4818)      │    727,518 │ flatten[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,359,918 (5.19 MB)

 Trainable params: 1,359,918 (5.19 MB)

 Non-trainable params: 0 (0.00 B)


Training Model with Attention Layer...
Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.0336 - loss: 7.0750 - val_accuracy: 0.0375 - val_loss: 6.6774
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0459 - loss: 6.4390 - val_accuracy: 0.0492 - val_loss: 6.6820
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.0563 - loss: 6.1982 - val_accuracy: 0.0536 - val_loss: 6.7138
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0645 - loss: 6.0119 - val_accuracy: 0.0618 - val_loss: 6.7796

Model with Attention Test Accuracy: 0.0375
